In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F
import torch.utils as utils
import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import torchvision.datasets as dsets
from torchvision import models
from torchvision.utils import save_image
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter('runs/thesis06_5w_1s')


from tqdm import tqdm

from flows import PlanarFlow
from utils import Binarize
from codes import Linear_flipout, Flatten, count_parameters, EfficientNet

from torchmeta.datasets import Omniglot, CIFARFS
from torchmeta.transforms import Categorical, ClassSplitter, Rotation
from torchvision.transforms import Compose, Resize, ToTensor
from torchmeta.utils.data import BatchMetaDataLoader


#from __future__ import print_function
import argparse
import cv2
import matplotlib.pyplot as plt

import os
cur_dir = "C:/Users/KJH/OneDrive - skku.edu/KJH/Projects/2019winter_research"
#cur_dir = "C:/Users/KJH-Laptop/OneDrive - skku.edu/KJH/Projects/2019winter_research/"
os.chdir(cur_dir)
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

import time
import copy
import random as rd

device = torch.device('cuda')

class net(nn.Module):
    def __init__(self, num_classes):
        super(net, self).__init__()
        self.input_dim = [1, 28, 28]
        self.num_classes = num_classes
        
        self.ctx = torch.hub.load('rwightman/gen-efficientnet-pytorch', 'efficientnet_b0', pretrained=True)
        self.ctx.conv_stem = nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        self.ctx.classifier = nn.Identity()
        
        self.layer = nn.LSTM(input_size=1280, hidden_size=1280, num_layers=5, bias=True, batch_first = True).to(device)
        self.dec_mu = nn.ModuleList()
        self.dec_logvar = nn.ModuleList()
        
        self.num_params = [[200 * 200, 200], [200 * 200, 200], [200 * self.num_classes, self.num_classes]]
        
        for layer_size in self.num_params:
            self.dec_mu.append(
                nn.Sequential(
                    nn.ELU(),
                    nn.Linear(1280, 1280, bias = True),
                    nn.ELU(),
                    nn.Linear(1280, layer_size[0] + layer_size[1], bias = True),
                ).to(device))
            
            self.dec_logvar.append(
                nn.Sequential(
                    nn.ELU(),
                    nn.Linear(1280, 1280, bias = True),
                    nn.ELU(),
                    nn.Linear(1280, layer_size[0] + layer_size[1], bias = True),
                ).to(device))
        
        self.encoder = nn.Sequential(
            nn.BatchNorm2d(1),
            Flatten(),            
            nn.Linear(784, 200, bias = False),
            nn.ELU()
        )

            
            
    def forward(self, input_train, label_train, input_test, adapt_lr, adapt_step = 1):
        ctx = self.ctx(input_train).view(input_train.shape[0], self.num_classes, 256)
        h = torch.stack([ctx[torch.where(label_train == x)].mean(dim = 0) for x in range(self.num_classes)], dim = 1)
        h = h.view(self.num_classes, 1, -1)
        
        params = nn.ParameterList()
        param_vals = []
        kld = torch.tensor(0., device=device)
        c = torch.empty([5, 1, 1280], requires_grad = False, device = device).fill_(0)
        x_train_init = self.encoder(input_train)
        x_train = x_train_init

        for ind, (dec_mu, dec_logvar) in enumerate(zip(self.dec_mu, self.dec_logvar)):
            x = torch.empty([1, 1, 1280], requires_grad=False, device = device).normal_(0, 1)
            f, (h, c) = self.layer(x, (h, c))
            
            mu = dec_mu(f.view(1, -1)).squeeze()
            logvar = dec_logvar(f.view(1, -1)).squeeze()
            params.append(nn.Parameter(torch.stack((mu, logvar), dim = 0), requires_grad = True))
        
        optimizer = optim.SGD(params, lr = adapt_lr)
        
        for step in range(adapt_step):
            x_train = x_train_init
            kld = torch.tensor(0., device=device)
            for ind, param in enumerate(params):
                param.retain_grad()

                weight_mu = param[0, :self.num_params[ind][0]].view(-1, self.num_params[ind][1])
                bias_mu = param[0, self.num_params[ind][0]:]

                weight_logvar = param[1, :self.num_params[ind][0]].view(-1, self.num_params[ind][1])
                bias_logvar = param[1, self.num_params[ind][0]:]

                weight_noise = torch.empty(weight_mu.shape, requires_grad = False, device = device).normal_(0,1)
                bias_noise = torch.empty(bias_mu.shape, requires_grad = False, device = device).normal_(0,1)
                in_sign = torch.empty(x_train.shape, requires_grad = False, device = device).uniform_(-1,1).sign()
                out_sign = torch.empty([x_train.shape[0], self.num_params[ind][1]], requires_grad = False, device = device).uniform_(-1,1).sign()

                x_train = torch.mm(x_train, weight_mu) + torch.mm(in_sign * x_train, weight_noise * weight_mu * weight_logvar.div(2).exp()) * out_sign
                x_train += (1 + bias_noise * bias_logvar.div(2).exp()) * bias_mu
                x_train = F.elu(x_train)

                kld += (mu.pow(2) - logvar + logvar.exp() - 1).mean()/2
                
            if step < adapt_step:
                optimizer.zero_grad()
                loss = F.cross_entropy(x_train, label_train) + 1e-6 * kld
                loss.backward(retain_graph = True)
                optimizer.step()
                
                
        x_test_init = self.encoder(input_test)
        x_test = x_test_init
        kld = torch.tensor(0., device=device)
        for ind, param in enumerate(params):
            param.retain_grad()

            weight_mu = param[0, :self.num_params[ind][0]].view(-1, self.num_params[ind][1])
            bias_mu = param[0, self.num_params[ind][0]:]

            weight_logvar = param[1, :self.num_params[ind][0]].view(-1, self.num_params[ind][1])
            bias_logvar = param[1, self.num_params[ind][0]:]

            weight_noise = torch.empty(weight_mu.shape, requires_grad = False, device = device).normal_(0,1)
            bias_noise = torch.empty(bias_mu.shape, requires_grad = False, device = device).normal_(0,1)
            in_sign = torch.empty(x_test.shape, requires_grad = False, device = device).uniform_(-1,1).sign()
            out_sign = torch.empty([x_test.shape[0], self.num_params[ind][1]], requires_grad = False, device = device).uniform_(-1,1).sign()

            x_test = torch.mm(x_test, weight_mu) + torch.mm(in_sign * x_test, weight_noise * weight_mu * weight_logvar.div(2).exp()) * out_sign
            x_test += (1 + bias_noise * bias_logvar.div(2).exp()) * bias_mu
            x_test = F.elu(x_test)

            kld += (mu.pow(2) - logvar + logvar.exp() - 1).mean()/2

        return x_test, kld

In [2]:
batch_size = 16
meta_trainset = Omniglot('./data/',
                   # Number of ways
                   num_classes_per_task=5,
                   # Resize the images to 28x28 and converts them to PyTorch tensors (from Torchvision)
                   transform=Compose([Resize(28), ToTensor()]),
                   # Transform the labels to integers (e.g. ("Glagolitic/character01", "Sanskrit/character14", ...) to (0, 1, ...))
                   target_transform=Categorical(num_classes=5),
                   # Creates new virtual classes with rotated versions of the images (from Santoro et al., 2016)
                   class_augmentations=[Rotation([90, 180, 270])],
                   meta_train=True,
                   download=True)
meta_trainset = ClassSplitter(meta_trainset, shuffle=True, num_train_per_class=5, num_test_per_class=15)
meta_trainloader = BatchMetaDataLoader(meta_trainset, batch_size=batch_size, num_workers=0)

model = net(5).cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

num_batches = 200
for batch_idx, meta_train_batch in zip(range(num_batches), meta_trainloader):
    start = time.time()

    train_inputs, train_targets = [x.to(device) for x in meta_train_batch["train"]]
    test_inputs, test_targets = [x.to(device) for x in meta_train_batch["test"]]
    
    cum_loss = torch.tensor(0., device=device)
    accuracy = torch.tensor(0., device=device)
    reg = torch.tensor(0., device=device)

    for task_idx, (train_input, train_target, test_input, test_target) in enumerate(
        zip(train_inputs, train_targets, test_inputs, test_targets)):
        optimizer.zero_grad()
        pred, kld = model(train_input, train_target, test_input, 1, 1)
        loss = criterion(pred, test_target)
        (loss + 1e-6 * kld).backward()
        optimizer.step()
        with torch.no_grad():
            cum_loss += loss
            accuracy += torch.sum(pred.argmax(1) == test_target.cuda())
            reg += kld
    cum_loss /= batch_size
    accuracy /= batch_size * 75
    reg /= batch_size

    writer.add_scalar('train/accuracy', accuracy, batch_idx)
    writer.add_scalar('train/reg', reg, batch_idx)
    if batch_idx % 10 == 0:
        print("%3d) loss = %f, kld = %f, acc = %f, time = %.3f sec" %(batch_idx, cum_loss, reg, accuracy, time.time() - start))

Using cache found in C:\Users\KJH/.cache\torch\hub\rwightman_gen-efficientnet-pytorch_master


  0) loss = 1.118820, kld = 0.000863, acc = 0.760000, time = 28.576 sec
 10) loss = 0.986487, kld = 0.000395, acc = 0.692500, time = 28.577 sec
 20) loss = 0.763360, kld = 0.000212, acc = 0.793333, time = 28.744 sec
 30) loss = 0.499484, kld = 0.000131, acc = 0.840000, time = 28.278 sec
 40) loss = 0.940768, kld = 0.000093, acc = 0.784167, time = 28.417 sec
 50) loss = 0.510337, kld = 0.000075, acc = 0.870833, time = 27.220 sec
 60) loss = 1.484791, kld = 0.000065, acc = 0.726667, time = 27.682 sec
 70) loss = 0.431145, kld = 0.000057, acc = 0.858333, time = 27.924 sec
 80) loss = 0.738859, kld = 0.000055, acc = 0.813333, time = 27.861 sec
 90) loss = 0.621747, kld = 0.000053, acc = 0.803333, time = 28.043 sec
100) loss = 0.665669, kld = 0.000052, acc = 0.798333, time = 27.888 sec
110) loss = 0.600377, kld = 0.000051, acc = 0.845000, time = 27.758 sec
120) loss = 0.502656, kld = 0.000051, acc = 0.860833, time = 28.028 sec
130) loss = 1.088834, kld = 0.000049, acc = 0.778333, time = 28.

In [3]:
optimizer = optim.Adam(model.parameters(), lr=1e-5)

num_batches = 200
for batch_idx, meta_train_batch in zip(range(num_batches), meta_trainloader):
    start = time.time()

    train_inputs, train_targets = [x.to(device) for x in meta_train_batch["train"]]
    test_inputs, test_targets = [x.to(device) for x in meta_train_batch["test"]]
    
    cum_loss = torch.tensor(0., device=device)
    accuracy = torch.tensor(0., device=device)
    reg = torch.tensor(0., device=device)

    for task_idx, (train_input, train_target, test_input, test_target) in enumerate(
        zip(train_inputs, train_targets, test_inputs, test_targets)):
        optimizer.zero_grad()
        pred, kld = model(train_input, train_target, test_input, 1, 1)
        loss = criterion(pred, test_target)
        (loss + 1e-6 * kld).backward()
        optimizer.step()
        with torch.no_grad():
            cum_loss += loss
            accuracy += torch.sum(pred.argmax(1) == test_target.cuda())
            reg += kld
    cum_loss /= batch_size
    accuracy /= batch_size * 75
    reg /= batch_size

    writer.add_scalar('train/accuracy', accuracy, batch_idx)
    writer.add_scalar('train/reg', reg, batch_idx)
    if batch_idx % 10 == 0:
        print("%3d) loss = %f, kld = %f, acc = %f, time = %.3f sec" %(batch_idx, cum_loss, reg, accuracy, time.time() - start))

  0) loss = 0.506735, kld = 0.000049, acc = 0.892500, time = 26.918 sec
 10) loss = 0.407875, kld = 0.000047, acc = 0.880833, time = 27.400 sec
 20) loss = 0.511529, kld = 0.000048, acc = 0.850833, time = 27.222 sec
 30) loss = 0.850249, kld = 0.000047, acc = 0.794167, time = 27.018 sec
 40) loss = 0.565616, kld = 0.000049, acc = 0.837500, time = 26.760 sec
 50) loss = 0.417291, kld = 0.000048, acc = 0.880833, time = 26.750 sec
 60) loss = 0.586819, kld = 0.000049, acc = 0.834167, time = 28.297 sec
 70) loss = 0.483006, kld = 0.000047, acc = 0.850833, time = 26.863 sec
 80) loss = 0.566484, kld = 0.000048, acc = 0.855833, time = 26.644 sec
 90) loss = 0.285287, kld = 0.000048, acc = 0.929167, time = 27.441 sec
100) loss = 0.838484, kld = 0.000050, acc = 0.856667, time = 26.387 sec
110) loss = 0.455802, kld = 0.000047, acc = 0.850833, time = 27.261 sec
120) loss = 1.181624, kld = 0.000050, acc = 0.815000, time = 27.434 sec
130) loss = 0.543299, kld = 0.000049, acc = 0.831667, time = 27.

In [4]:
optimizer = optim.Adam(model.parameters(), lr=1e-7)

num_batches = 200
for batch_idx, meta_train_batch in zip(range(num_batches), meta_trainloader):
    start = time.time()

    train_inputs, train_targets = [x.to(device) for x in meta_train_batch["train"]]
    test_inputs, test_targets = [x.to(device) for x in meta_train_batch["test"]]
    
    cum_loss = torch.tensor(0., device=device)
    accuracy = torch.tensor(0., device=device)
    reg = torch.tensor(0., device=device)

    for task_idx, (train_input, train_target, test_input, test_target) in enumerate(
        zip(train_inputs, train_targets, test_inputs, test_targets)):
        optimizer.zero_grad()
        pred, kld = model(train_input, train_target, test_input, 1, 1)
        loss = criterion(pred, test_target)
        (loss + 1e-6 * kld).backward()
        optimizer.step()
        with torch.no_grad():
            cum_loss += loss
            accuracy += torch.sum(pred.argmax(1) == test_target.cuda())
            reg += kld
    cum_loss /= batch_size
    accuracy /= batch_size * 75
    reg /= batch_size

    writer.add_scalar('train/accuracy', accuracy, batch_idx)
    writer.add_scalar('train/reg', reg, batch_idx)
    if batch_idx % 10 == 0:
        print("%3d) loss = %f, kld = %f, acc = %f, time = %.3f sec" %(batch_idx, cum_loss, reg, accuracy, time.time() - start))

  0) loss = 1.147690, kld = 0.000048, acc = 0.868333, time = 28.237 sec
 10) loss = 0.157103, kld = 0.000047, acc = 0.959167, time = 26.679 sec
 20) loss = 0.371660, kld = 0.000048, acc = 0.888333, time = 26.710 sec
 30) loss = 0.850984, kld = 0.000048, acc = 0.885833, time = 26.666 sec
 40) loss = 0.423712, kld = 0.000048, acc = 0.912500, time = 26.874 sec
 50) loss = 0.658808, kld = 0.000050, acc = 0.812500, time = 26.856 sec
 60) loss = 0.478345, kld = 0.000049, acc = 0.840833, time = 27.026 sec
 70) loss = 0.696860, kld = 0.000049, acc = 0.802500, time = 27.617 sec
 80) loss = 0.639119, kld = 0.000047, acc = 0.855833, time = 27.302 sec
 90) loss = 0.444986, kld = 0.000049, acc = 0.879167, time = 26.494 sec
100) loss = 0.531235, kld = 0.000047, acc = 0.857500, time = 26.444 sec
110) loss = 2.056088, kld = 0.000049, acc = 0.801667, time = 28.118 sec
120) loss = 0.506234, kld = 0.000048, acc = 0.883333, time = 27.576 sec
130) loss = 0.518983, kld = 0.000047, acc = 0.893333, time = 27.

In [7]:
meta_testset  = Omniglot('./data/',
                   # Number of ways
                   num_classes_per_task=5,
                   # Resize the images to 28x28 and converts them to PyTorch tensors (from Torchvision)
                   transform=Compose([Resize(28), ToTensor()]),
                   # Transform the labels to integers (e.g. ("Glagolitic/character01", "Sanskrit/character14", ...) to (0, 1, ...))
                   target_transform=Categorical(num_classes=5),
                   # Creates new virtual classes with rotated versions of the images (from Santoro et al., 2016)
                   class_augmentations=[Rotation([90, 180, 270])],
                   meta_train=False,
                   download=True)
meta_testset = ClassSplitter(meta_testset, shuffle=True, num_train_per_class=5, num_test_per_class=15)
meta_testloader = BatchMetaDataLoader(meta_testset, batch_size=batch_size, num_workers=0)

tot_loss = torch.tensor(0., device=device)
tot_acc = torch.tensor(0., device=device)
tot_reg = torch.tensor(0., device=device)

for batch_idx, meta_test_batch in zip(range(num_batches), meta_testloader):
    start = time.time()
    train_inputs, train_targets = [x.to(device) for x in meta_test_batch["train"]]
    test_inputs, test_targets = [x.to(device) for x in meta_test_batch["test"]]
    
    cum_loss = torch.tensor(0., device=device)
    accuracy = torch.tensor(0., device=device)
    reg = torch.tensor(0., device=device)

    for task_idx, (train_input, train_target, test_input, test_target) in enumerate(
        zip(train_inputs, train_targets, test_inputs, test_targets)):
        optimizer.zero_grad()
        pred, kld = model(train_input, train_target, test_input, 1, 1)
        with torch.no_grad():            
            loss = criterion(pred, test_target)
            cum_loss += loss
            accuracy += torch.sum(pred.argmax(1) == test_target.cuda())
            reg += kld

    tot_loss += cum_loss / batch_size
    tot_acc += accuracy  / (batch_size * 75)
    tot_reg += reg / batch_size
        
tot_loss /= num_batches
tot_acc /= num_batches
tot_reg /= num_batches
        
print("loss = %f, test_kld = %f, meta_test_acc = %f, time = %.3f sec" %(tot_loss, tot_reg, tot_acc, time.time() - start))

loss = 0.597349, test_kld = 0.000049, meta_test_acc = 0.857650, time = 19.940 sec


In [6]:
torch.save(model.state_dict(), "./save/thesis06_omniglot5w5s_5step")